Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [3]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (18724, 28, 28) (18724,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (18724, 28, 28, 1) (18724, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [6]:
batch_size = 16
patch_size = 5 #patch/kernel
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))

  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))

  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))

  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    #tf.nn.conv2d args = input(tensor), filter(tensor), strides, padding
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [8]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 100 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.355869
Minibatch accuracy: 37.5%
Validation accuracy: 100.0%
Minibatch loss at step 100: 0.094340
Minibatch accuracy: 100.0%
Validation accuracy: 87.5%
Minibatch loss at step 200: 0.612656
Minibatch accuracy: 75.0%
Validation accuracy: 84.5%
Minibatch loss at step 300: 0.153626
Minibatch accuracy: 93.8%
Validation accuracy: 78.7%
Minibatch loss at step 400: 0.285713
Minibatch accuracy: 87.5%
Validation accuracy: 79.7%
Minibatch loss at step 500: 0.540596
Minibatch accuracy: 81.2%
Validation accuracy: 88.1%
Minibatch loss at step 600: 0.208195
Minibatch accuracy: 93.8%
Validation accuracy: 86.3%
Minibatch loss at step 700: 0.497474
Minibatch accuracy: 81.2%
Validation accuracy: 88.1%
Minibatch loss at step 800: 0.574290
Minibatch accuracy: 81.2%
Validation accuracy: 88.5%
Minibatch loss at step 900: 0.535384
Minibatch accuracy: 87.5%
Validation accuracy: 86.7%
Minibatch loss at step 1000: 0.377708
Minibatch accuracy: 81.2%
Validation accuracy: 81.

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

Max pooling usage in convolutional model. Replaces the convolutions with a max_pooling operation instead.

In [30]:
batch_size = 16
patch_size = 5 #patch/kernel
depth = 16
num_hidden = 64

#tf.nn.max_pool() performs the pooling operation
#args = value(4D tensor), ksize(A list of ints >= 4. The size of the input for each dimension
# of the tensor), strides(A list of ints that has length >= 4. The stride of the sliding window
#for each dimension of the tensor), padding algorithm is either 'SAME' or 'VALID'
#value(tensor), ksize(list of ints for each dimension of tensor), strides, algorithm

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))

  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))

  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))

  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    #tf.nn.conv2d args = input(tensor), filter(tensor), strides, padding
    #conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    #hidden = tf.nn.relu(conv + layer1_biases)
    
    max_pool = tf.nn.max_pool(data,[1,2,2,1],[1,2,2,1],padding='SAME')
    hidden = tf.nn.relu(max_pool + layer1_biases)

    #conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    #hidden = tf.nn.relu(conv + layer1_biases)
    
    max_pool = tf.nn.max_pool(hidden, [1,2,2,1], [1,2,2,1], padding='SAME')
    hidden = tf.nn.relu(max_pool + layer2_biases)
    
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [31]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 100 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.287970
Minibatch accuracy: 37.5%
Validation accuracy: 100.0%
Minibatch loss at step 100: 1.411181
Minibatch accuracy: 12.5%
Validation accuracy: 100.0%
Minibatch loss at step 200: 1.281141
Minibatch accuracy: 43.8%
Validation accuracy: 0.0%
Minibatch loss at step 300: 1.853382
Minibatch accuracy: 37.5%
Validation accuracy: 0.0%
Minibatch loss at step 400: 1.230878
Minibatch accuracy: 25.0%
Validation accuracy: 0.0%
Minibatch loss at step 500: 1.452644
Minibatch accuracy: 25.0%
Validation accuracy: 0.0%
Minibatch loss at step 600: 1.248930
Minibatch accuracy: 62.5%
Validation accuracy: 0.0%
Minibatch loss at step 700: 1.805952
Minibatch accuracy: 18.8%
Validation accuracy: 0.0%
Minibatch loss at step 800: 1.466969
Minibatch accuracy: 37.5%
Validation accuracy: 0.0%
Minibatch loss at step 900: 1.352846
Minibatch accuracy: 25.0%
Validation accuracy: 0.0%
Minibatch loss at step 1000: 0.971371
Minibatch accuracy: 81.2%
Validation accuracy: 69.2%
Test 

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [48]:
batch_size = 16
patch_size = 5 #patch/kernel
depth = 16
num_hidden = 64

#tf.nn.max_pool() performs the pooling operation
#args = value(4D tensor), ksize(A list of ints >= 4. The size of the input for each dimension
# of the tensor), strides(A list of ints that has length >= 4. The stride of the sliding window
#for each dimension of the tensor), padding algorithm is either 'SAME' or 'VALID'
#value(tensor), ksize(list of ints for each dimension of tensor), strides, algorithm

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))

  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))

  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))

  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    #tf.nn.conv2d args = input(tensor), filter(tensor), strides, padding
    
    # first layer = convolution -> relu -> max_pool
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    max_pool = tf.nn.max_pool(hidden,[1,2,2,1],[1,2,2,1],padding='SAME')
    
    #second layer = max_pool -> convolution -> relu -> max_pool
    conv = tf.nn.conv2d(max_pool, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    max_pool = tf.nn.max_pool(hidden, [1,2,2,1], [1,2,2,1], padding='SAME')
    
    #third layer = fully connected layer
    shape = max_pool.get_shape().as_list()
    reshape = tf.reshape(max_pool, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  # L2 regularization
  B = .003
  loss = loss + B*.5*(tf.nn.l2_loss(layer1_weights) + tf.nn.l2_loss(layer2_weights) /
                     tf.nn.l2_loss(layer3_weights) + tf.nn.l2_loss(layer4_weights))


  # Learning rate decay
  starter_learning_rate = .05
  # number of steps taken
  globalstep = tf.Variable(0) # count the number of steps taken
  learning_rate = tf.train.exponential_decay(starter_learning_rate,
                                             globalstep, 200, .95, staircase=True)
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss,global_step = globalstep)  
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [49]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 100 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.927149
Minibatch accuracy: 0.0%
Validation accuracy: 0.0%
Minibatch loss at step 100: 0.210699
Minibatch accuracy: 100.0%
Validation accuracy: 89.8%
Minibatch loss at step 200: 0.620763
Minibatch accuracy: 75.0%
Validation accuracy: 83.2%
Minibatch loss at step 300: 0.072872
Minibatch accuracy: 100.0%
Validation accuracy: 81.6%
Minibatch loss at step 400: 0.318307
Minibatch accuracy: 87.5%
Validation accuracy: 80.1%
Minibatch loss at step 500: 0.402667
Minibatch accuracy: 81.2%
Validation accuracy: 89.0%
Minibatch loss at step 600: 0.148513
Minibatch accuracy: 93.8%
Validation accuracy: 87.5%
Minibatch loss at step 700: 0.337639
Minibatch accuracy: 87.5%
Validation accuracy: 88.3%
Minibatch loss at step 800: 0.498603
Minibatch accuracy: 81.2%
Validation accuracy: 89.0%
Minibatch loss at step 900: 0.768708
Minibatch accuracy: 87.5%
Validation accuracy: 88.1%
Minibatch loss at step 1000: 0.325025
Minibatch accuracy: 81.2%
Validation accuracy: 82.0%